In [1]:
from datasets import load_dataset

ds = load_dataset("hackercupai/hackercup")
ds

DatasetDict({
    sample: Dataset({
        features: ['name', 'year', 'round', 'statement', 'input', 'solution', 'code', 'output', 'sample_input', 'sample_output', 'images'],
        num_rows: 10
    })
    full: Dataset({
        features: ['name', 'year', 'round', 'statement', 'input', 'solution', 'code', 'output', 'sample_input', 'sample_output', 'images'],
        num_rows: 284
    })
})

In [18]:
import io
import sys
import traceback
import contextlib
from unittest.mock import patch

def run_extracted_code(extracted_code, test_input):
    output = io.StringIO()
    error = None
    test_input_lines = [line.strip() for line in test_input.strip().split('\n') if line.strip()]

    def mock_input():
        if not test_input_lines:
            raise ValueError("Not enough input data provided")
        return test_input_lines.pop(0)

    local_scope = {'__name__': '__main__'}
    
    with patch('builtins.input', mock_input), contextlib.redirect_stdout(output):
        try:
            # Compile the code object
            code_obj = compile(extracted_code, '<string>', 'exec')
            
            # Execute the compiled code
            exec(code_obj, local_scope)
            
        except Exception as e:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            
            # Get the line number where the error occurred
            tb = traceback.extract_tb(exc_traceback)
            line_no = tb[-1].lineno
            
            # Get the line of code that caused the error
            code_lines = extracted_code.split('\n')
            error_line = code_lines[line_no - 1] if line_no <= len(code_lines) else "Unknown"
            
            error = f"Error occurred on line {line_no}:\n"
            error += f"Code: {error_line.strip()}\n"
            error += f"Exception: {exc_type.__name__}: {str(exc_value)}\n"
            error += "Traceback:\n"
            error += ''.join(traceback.format_tb(exc_traceback))

    return output.getvalue(), error


# Example usage
extracted_code = """
def max_decker_cheeseburgers(A, B, C):
    n = A + B * 2
    dp = [0] * (n + 1)
    for i in range(1, n + 1):
        if i >= A:
            dp[i] = max(dp[i], dp[i - A] + 1)
        if i >= B * 2:
            dp[i] = max(dp[i], dp[i - B * 2] + 2)
    return dp[n]

def solve_problem():
    T = int(input())
    results = []
    for t in range(1, T + 1):
        A = int(input())
        B = int(input())
        C = int(input())
        result = max_decker_cheeseburgers(A, B, C)
        results.append(f'Case #{t}: {result}')
    for result in results:
        print(result)
    
if __name__ == "__main__":
    solve_problem()
"""


test_input = """
6
2 3 5
2 3 2
2 3 1
5 1 100
1 3 100
1 1 1000000000000

"""

output, error = run_extracted_code(extracted_code, test_input)
if error:
    print("Error detected:")
    print(error)
else:
    print("Output:")
    print(output)

Error detected:
Error occurred on line 16:
Code: A = int(input())
Exception: ValueError: invalid literal for int() with base 10: '2 3 5'
Traceback:
  File "C:\Users\PC\AppData\Local\Temp\ipykernel_21784\480169526.py", line 25, in run_extracted_code
    exec(code_obj, local_scope)
  File "<string>", line 25, in <module>
  File "<string>", line 16, in solve_problem



In [43]:
code_solution = {
  "solution_code": {
    "language": "Python",
    "error_line": "if __name__ == '__main__':",
    "code": "def max_k_deck_burgers(C):\n    return C // 2\n\ndef main():\n    T = int(input())\n    results = []\n    for _ in range(T):\n        C = int(input())\n        results.append(f\"Case # {_ + 1}: {max_k_deck_burgers(C)}\")\n    print('\\n'.join(results))\n\nif __name__ == '__main__':\n    main()",
    "improvement": "Ensured the `if __name__ == '__main__':` block is correctly placed to execute the main function when the script is run directly. No changes were made to the `input()` function usage as per the guidelines."
  }
}

def extract_python_code(response):
    return response

In [46]:
generated_code = extract_python_code(code_solution['solution_code']['code'])
generated_code

'def max_k_deck_burgers(C):\n    return C // 2\n\ndef main():\n    T = int(input())\n    results = []\n    for _ in range(T):\n        C = int(input())\n        results.append(f"Case # {_ + 1}: {max_k_deck_burgers(C)}")\n    print(\'\\n\'.join(results))\n\nif __name__ == \'__main__\':\n    main()'

In [68]:
def check_code_structure(extracted_code):
    # Check if the phrase "__name__ == '__main__'" or '__name__ == "__main__"' is present in the code
    if "__name__ == '__main__'" not in extracted_code:
        return False, "Missing `if __name__ == '__main__':` block."
    
    return True, None

In [70]:
extracted_code = """
def main():
    import sys
    input = sys.stdin.read
    data = input().split()
    num_tests = int(data[0])
    results = []
    index = 1
    for _ in range(num_tests):
        budget = int(data[index])
        index += 1
        result = calculate_max_decks(budget)
        results.append(f"Case #{len(results) + 1}: {result}")
    print('\n'.join(results))

if __name__ == '__main__':
    main()
"""

In [72]:
check_code_structure(extracted_code)

Extracted code: 
def main():
    import sys
    input = sys.stdin.read
    data = input().split()
    num_tests = int(data[0])
    results = []
    index = 1
    for _ in range(num_tests):
        budget = int(data[index])
        index += 1
        result = calculate_max_decks(budget)
        results.append(f"Case #{len(results) + 1}: {result}")
    print('
'.join(results))

if __name__ == '__main__':
    main()



(True, None)